# Data Preperation for Streamlit WW2 Implementation
Our WW2 data contains attacks on individual coordinates.

In this session we want to create a nice looking web-page that let's us easily compare statistics between individual countries.

In order to do this we have to aggregate our individual coordinate-based data to per-country data.

## Imports

In [1]:
import pandas as pd
import geopandas as gpd

## Load Raw Data

In [2]:
# load data on european countries
europe = pd.read_pickle("../data/europe.p").to_crs("EPSG:4326")
# contains all countries intersecting our polygon from the WW2 analysis
europe.sample(5)# not only european countries per se
# columns of interest for us: name and geometry
europe.head()


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((45.00000 42.60827, 44.53762 42..."
1,5320045,Europe,Norway,-99,364700.0,"POLYGON ((31.10104 69.55810, 29.39955 69.15692..."
2,57713,North America,Greenland,GRL,2173.0,"POLYGON ((-20.72922 73.50000, -20.76234 73.464..."
3,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((8.74601 42.62812, 9.39000 43.0..."
4,8299706,Asia,Israel,ISR,297000.0,"POLYGON ((35.05295 33.00000, 35.09846 33.08054..."


In [3]:
# load WW2 data for target_df
# geometry contains coordinate POINTS
europe_data = pd.read_pickle("../data/gdf_europe.p").to_crs("EPSG:4326")
europe_data.head()

,country code,country,name of powerplant,capacity in MW,latitude,longitude,primary_fuel,start date,owner of plant,geolocation_source,estimated_generation_gwh_2020,green,continent,geometry
9,ALB,Albania,Bistrica 1,27.0,39.9116,20.1047,Hydro,1965,NaN,GEODB,88.45,True,Europe,POINT (20.10470 39.91160)
10,ALB,Albania,Fierza,500.0,42.2514,20.0431,Hydro,1978,NaN,GEODB,1648.24,True,Europe,POINT (20.04310 42.25140)
11,ALB,Albania,Koman,600.0,42.1033,19.8224,Hydro,1985,NaN,GEODB,1982.72,True,Europe,POINT (19.82240 42.10330)
12,ALB,Albania,Lanabregas,5.0,41.3428,19.8964,Hydro,1951,NaN,GEODB,15.23,True,Europe,POINT (19.89640 41.34280)
13,ALB,Albania,Shkopet,24.0,41.6796,19.8305,Hydro,1963,NaN,GEODB,83.57,True,Europe,POINT (19.83050 41.67960)


## Replace target coordinates by target countries polygons

In [4]:
res_intersect_europe = gpd.overlay(europe_data, europe, how='intersection')
# Basically checks for each geometry in "europe_data" (target coordinates) 
# if intersects with geometry of "target_df" (country)
# -> Merges both dataframes and keeps country polygon instead of target coordinates
res_intersect_europe.head()

,country code,country,name of powerplant,capacity in MW,latitude,longitude,primary_fuel,start date,owner of plant,geolocation_source,estimated_generation_gwh_2020,green,continent_1,pop_est,continent_2,name,iso_a3,gdp_md_est,geometry
0,ALB,Albania,Bistrica 1,27.0,39.9116,20.1047,Hydro,1965,NaN,GEODB,88.45,True,Europe,3047987,Europe,Albania,ALB,33900.0,POINT (20.10470 39.91160)
1,ALB,Albania,Fierza,500.0,42.2514,20.0431,Hydro,1978,NaN,GEODB,1648.24,True,Europe,3047987,Europe,Albania,ALB,33900.0,POINT (20.04310 42.25140)
2,ALB,Albania,Koman,600.0,42.1033,19.8224,Hydro,1985,NaN,GEODB,1982.72,True,Europe,3047987,Europe,Albania,ALB,33900.0,POINT (19.82240 42.10330)
3,ALB,Albania,Lanabregas,5.0,41.3428,19.8964,Hydro,1951,NaN,GEODB,15.23,True,Europe,3047987,Europe,Albania,ALB,33900.0,POINT (19.89640 41.34280)
4,ALB,Albania,Shkopet,24.0,41.6796,19.8305,Hydro,1963,NaN,GEODB,83.57,True,Europe,3047987,Europe,Albania,ALB,33900.0,POINT (19.83050 41.67960)


In [5]:
# Set index to country name so we can directly insert data grouped by country name
europe.set_index("name", inplace=True, drop=False)

In [6]:
def insert_totals(target_df, data_df):
    # get attack counts and total weight of explosives per year
    # and apply it to a given target country
    # get totals
    target_df[f"N_powerplants"] = data_df.groupby("name").size()
    target_df[f"N_green"] = data_df.loc[data_df["green"]].groupby("name").size()
    # replace NaN (no a number) values with 0
    target_df.fillna(0, inplace=True)
    n_green = data_df.groupby(["name", "green"])["name"].size().unstack()
    target_df["green_ratio"] = (n_green[True] / n_green.sum(axis=1)).fillna(0)
    return target_df

In [7]:
europe = insert_totals(europe, res_intersect_europe)

In [8]:
europe = europe.loc[europe["continent"] == "Europe"]

In [26]:
def insert_fuel_types(target_df, data_df):
    for pp_type in data_df["primary_fuel"].unique():
        target_df[f"N_fuel_{pp_type}"] = data_df.loc[data_df["primary_fuel"] == pp_type].groupby("name").size()
        data_df.loc[data_df["primary_fuel"] == pp_type, f"is_{pp_type}"] = True
        data_df.loc[data_df["primary_fuel"] != pp_type, f"is_{pp_type}"] = False 
        ratio = data_df.groupby(["name", f"is_{pp_type}"]).size().unstack().fillna(0)
        target_df[f"fuel_ratio_{pp_type}"] = ratio[True] / ratio.sum(axis=1)

        production = data_df.groupby(["name", f"is_{pp_type}"])["estimated_generation_gwh_2020"].sum().unstack().fillna(0)

        target_df[f"fuel_generation_ratio_{pp_type}"] = production[True] / production.sum(axis=1)
        target_df[f"fuel_generation_total_{pp_type}"] = production[True]
    return target_df

In [27]:
europe = insert_fuel_types(europe, res_intersect_europe).fillna(0)

# German Bundeslaender

In [41]:
# load data containing German Bundesländer
# similarly to the europe dataset it contains polygons mapping individual Bundesländer
bl = gpd.read_file('../data/vg2500_geo84.zip').to_crs("EPSG:4326")
# rename the column that specifies the name so that is the same with the remaining data
bl.rename({"GEN": "name"}, axis=1, inplace=True)

In [42]:
bl.head()

,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
0,2,02,020000000000,Hamburg,1.752538e+05,7.605398e+08,"MULTIPOLYGON (((10.19114 53.72299, 10.18210 53..."
1,2,03,030000000000,Niedersachsen,2.016496e+06,4.771641e+10,"MULTIPOLYGON (((8.68962 53.87999, 8.69297 53.8..."
2,2,04,040000000000,Bremen,1.549716e+05,4.054809e+08,"MULTIPOLYGON (((8.52976 53.21640, 8.55399 53.2..."
3,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,"POLYGON ((8.66673 52.52149, 8.70289 52.50053, ..."
4,2,06,060000000000,Hessen,1.105093e+06,2.109232e+10,"POLYGON ((9.50560 51.62777, 9.50842 51.62712, ..."


In [43]:
# same as before but now we want to aggregate data for individual Bundesländer
res_intersect_bl = gpd.overlay(europe_data, bl, how='intersection')

In [44]:
res_intersect_bl.sample(5)

,country code,country,name of powerplant,capacity in MW,latitude,longitude,primary_fuel,start date,owner of plant,geolocation_source,estimated_generation_gwh_2020,green,continent,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
603,DEU,Germany,Sietzsch Solar Power Plant,10.0,51.4896,12.1908,Solar,-1,NaN,Industry About,15.09,True,Europe,2,15,150000000000,Sachsen-Anhalt,9.490968e+05,2.049498e+10,POINT (12.19080 51.48960)
822,DEU,Germany,KW Rheinfelden,46.7,47.5699,7.8101,Hydro,2010,Energiedienst AG,Open Power System Data,133.24,True,Europe,2,08,080000000000,Baden-Württemberg,1.298892e+06,3.580140e+10,POINT (7.81010 47.56990)
138,DEU,Germany,Hurlach Solar Power Plant,4.3,48.1074,10.8538,Solar,-1,NaN,Industry About,5.91,True,Europe,2,09,090000000000,Bayern,2.046039e+06,7.055007e+10,POINT (10.85380 48.10740)
1021,DEU,Germany,Leinefelde-Worbis,8.0,51.3844,10.3364,Solar,-1,NaN,Wiki-Solar,11.22,True,Europe,2,16,160000000000,Thüringen,9.619427e+05,1.617853e+10,POINT (10.33640 51.38440)
1273,DEU,Germany,W�rrstadt Solar Power Plant,5.6,49.8289,8.1322,Solar,-1,NaN,Industry About,8.45,True,Europe,2,07,070000000000,Rheinland-Pfalz,9.106504e+05,1.983491e+10,POINT (8.13220 49.82890)


In [45]:
bl.set_index("name", inplace=True)
bl = insert_totals(bl, res_intersect_bl)
bl = insert_fuel_types(bl, res_intersect_bl)
bl

,USE,RS,RS_ALT,SHAPE_LENG,SHAPE_AREA,geometry,N_powerplants,N_green,green_ratio,N_fuel_Hydro,...,fuel_generation_ratio_Nuclear,fuel_generation_total_Nuclear,N_fuel_Other,fuel_ratio_Other,fuel_generation_ratio_Other,fuel_generation_total_Other,N_fuel_Wind,fuel_ratio_Wind,fuel_generation_ratio_Wind,fuel_generation_total_Wind
name,,,,,,,,,,,,,,,,,,,,,
Hamburg,2,02,020000000000,1.752538e+05,7.605398e+08,"MULTIPOLYGON (((10.19114 53.72299, 10.18210 53...",7,1,0.142857,NaN,...,0.000000,0.00,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.000000,0.00
Niedersachsen,2,03,030000000000,2.016496e+06,4.771641e+10,"MULTIPOLYGON (((8.68962 53.87999, 8.69297 53.8...",45,16,0.355556,2.0,...,0.463524,18588.72,1.0,0.022222,0.023785,953.84,NaN,0.000000,0.000000,0.00
Bremen,2,04,040000000000,1.549716e+05,4.054809e+08,"MULTIPOLYGON (((8.52976 53.21640, 8.55399 53.2...",8,1,0.125000,1.0,...,0.000000,0.00,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.000000,0.00
Nordrhein-Westfalen,2,05,050000000000,1.352108e+06,3.404727e+10,"POLYGON ((8.66673 52.52149, 8.70289 52.50053, ...",131,24,0.183206,5.0,...,0.000000,0.00,4.0,0.030534,0.003987,559.39,NaN,0.000000,0.000000,0.00
Hessen,2,06,060000000000,1.105093e+06,2.109232e+10,"POLYGON ((9.50560 51.62777, 9.50842 51.62712, ...",42,19,0.452381,3.0,...,0.000000,0.00,1.0,0.023810,0.005029,65.13,NaN,0.000000,0.000000,0.00
Rheinland-Pfalz,2,07,070000000000,9.106504e+05,1.983491e+10,"POLYGON ((7.84657 50.87861, 7.85901 50.87733, ...",72,53,0.736111,10.0,...,0.000000,0.00,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.000000,0.00
Baden-Württemberg,2,08,080000000000,1.298892e+06,3.580140e+10,"MULTIPOLYGON (((9.64634 49.77782, 9.63806 49.7...",95,59,0.621053,21.0,...,0.334574,19274.21,1.0,0.010526,0.003214,185.14,NaN,0.000000,0.000000,0.00
Bayern,2,09,090000000000,2.046039e+06,7.055007e+10,"POLYGON ((10.47207 47.56964, 10.47319 47.57081...",305,261,0.855738,63.0,...,0.508339,27264.80,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.000000,0.00
Saarland,2,10,100000000000,2.829108e+05,2.578542e+09,"POLYGON ((7.06853 49.62349, 7.08023 49.61196, ...",24,14,0.583333,NaN,...,0.000000,0.00,1.0,0.041667,0.024724,281.03,NaN,0.000000,0.000000,0.00


## German Landkreise
Let's go even lower than Bundesländer

In [49]:
kreise = gpd.read_file('../data/vg2500_krs.zip').to_crs("EPSG:4326")
kreise.rename({"GEN": "name"}, axis=1, inplace=True)
kreise.sample(5)

,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
312,4,09774,097740000000,Günzburg,151126.841941,7.660444e+08,"POLYGON ((10.42423 48.52114, 10.42234 48.51623..."
219,4,08426,084260000000,Biberach,214557.354650,1.414375e+09,"POLYGON ((9.89724 48.29384, 9.89752 48.28848, ..."
65,4,05112,051120000000,Duisburg,76480.458049,2.358640e+08,"POLYGON ((6.78424 51.53198, 6.78317 51.53050, ..."
161,4,07313,073130000000,Landau in der Pfalz,61094.274804,9.034076e+07,"MULTIPOLYGON (((8.12975 49.22772, 8.14057 49.2..."
366,4,15082,150820000000,Anhalt-Bitterfeld,268526.034558,1.449396e+09,"POLYGON ((12.31291 52.09361, 12.31393 52.09303..."


repeat the stepts

In [50]:
res_intersect_kreise = gpd.overlay(europe_data, kreise, how='intersection')

In [51]:
kreise.set_index("name", inplace=True)
kreise = insert_totals(kreise, res_intersect_kreise)
kreise = insert_fuel_types(kreise, res_intersect_kreise)
kreise

,USE,RS,RS_ALT,SHAPE_LENG,SHAPE_AREA,geometry,N_powerplants,N_green,green_ratio,N_fuel_Hydro,...,fuel_generation_ratio_Wind,fuel_generation_total_Wind,N_fuel_Oil,fuel_ratio_Oil,fuel_generation_ratio_Oil,fuel_generation_total_Oil,N_fuel_Nuclear,fuel_ratio_Nuclear,fuel_generation_ratio_Nuclear,fuel_generation_total_Nuclear
name,,,,,,,,,,,,,,,,,,,,,
Flensburg,4,01001,010010000000,32467.034276,5.324899e+07,"POLYGON ((9.48982 54.82296, 9.50467 54.81004, ...",1.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
Kiel,4,01002,010020000000,73027.302434,1.218654e+08,"POLYGON ((10.19057 54.38909, 10.18370 54.38910...",2.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
Lübeck,4,01003,010030000000,104861.094220,2.211386e+08,"POLYGON ((10.89297 53.95695, 10.90337 53.95705...",0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Neumünster,4,01004,010040000000,39858.818376,8.092363e+07,"POLYGON ((10.03454 54.12915, 10.02277 54.10938...",0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dithmarschen,4,01051,010510000000,202206.675707,1.450311e+09,"MULTIPOLYGON (((9.05176 54.36640, 9.07193 54.3...",3.0,3.0,1.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Saalfeld-Rudolstadt,4,16073,160730000000,209670.746788,1.023057e+09,"POLYGON ((11.42397 50.80862, 11.42072 50.80306...",2.0,1.0,0.5,1.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
Saale-Holzland-Kreis,4,16074,160740000000,172686.880125,8.176883e+08,"POLYGON ((12.01807 50.96757, 12.02085 50.94806...",0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Saale-Orla-Kreis,4,16075,160750000000,185679.957051,1.144877e+09,"POLYGON ((11.94482 50.59236, 11.92423 50.58608...",3.0,3.0,1.0,1.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0


In [57]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world_data = pd.read_pickle("../data/gdf_world.p").to_crs("EPSG:4326")
res_intersect_world = gpd.overlay(world_data, world, how='intersection')

In [61]:
world.set_index("name", inplace=True)
world = insert_totals(world, res_intersect_world)
world = insert_fuel_types(world, res_intersect_world)
world

,pop_est,continent,iso_a3,gdp_md_est,geometry,N_powerplants,N_green,green_ratio,N_fuel_Hydro,fuel_ratio_Hydro,...,fuel_generation_ratio_Storage,fuel_generation_total_Storage,N_fuel_Geothermal,fuel_ratio_Geothermal,fuel_generation_ratio_Geothermal,fuel_generation_total_Geothermal,N_fuel_Petcoke,fuel_ratio_Petcoke,fuel_generation_ratio_Petcoke,fuel_generation_total_Petcoke
name,,,,,,,,,,,,,,,,,,,,,
Fiji,920938,Oceania,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",10.0,8.0,0.800000,4.0,0.400000,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0
Tanzania,53950935,Africa,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",10.0,7.0,0.700000,6.0,0.600000,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0
W. Sahara,603253,Africa,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Canada,35623680,North America,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",1116.0,1013.0,0.907706,550.0,0.492832,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0
United States of America,326625791,North America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",9654.0,6028.0,0.624404,1421.0,0.147193,...,0.0,0.0,65.0,0.006733,0.004648,21022.26,11.0,0.001139,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Serbia,7111024,Europe,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",8.0,2.0,0.250000,2.0,0.250000,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0
Montenegro,642550,Europe,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009...",3.0,2.0,0.666667,2.0,0.666667,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0
Kosovo,1895250,Europe,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787...",4.0,0.0,0.000000,NaN,0.000000,...,0.0,0.0,NaN,0.000000,0.000000,0.00,NaN,0.000000,0.0,0.0


In [71]:
world["name"] = world.index
bl["name"] = bl.index
kreise["name"] = kreise.index

In [74]:
europe.fillna(0).to_pickle("../data/europe_pp.p")
bl.fillna(0).to_pickle("../data/bl_pp.p")
kreise.fillna(0).to_pickle("../data/kreise_pp.p")
world.fillna(0).to_pickle("../data/world_pp.p")
res_intersect_kreise.to_pickle("../data/kreise_full.p")
res_intersect_world.to_pickle("../data/world_full.p")

In [75]:
res_intersect_kreise

,country code,country,name of powerplant,capacity in MW,latitude,longitude,primary_fuel,start date,owner of plant,geolocation_source,...,is_Hydro,is_Solar,is_Gas,is_Biomass,is_Coal,is_Other,is_Waste,is_Wind,is_Oil,is_Nuclear
0,AUT,Austria,Braunau-Simbach,100.0,48.2440,13.0060,Hydro,1954,Verbund,WRI,...,True,False,False,False,False,False,False,False,False,False
1,DEU,Germany,Ering,36.5,48.2935,13.1585,Hydro,1942,Innwerk AG,Open Power System Data,...,True,False,False,False,False,False,False,False,False,False
2,DEU,Germany,Malching,4.8,48.2965,13.1589,Solar,-1,NaN,Wiki-Solar,...,False,True,False,False,False,False,False,False,False,False
3,DEU,Germany,K�ching Solar Power Plant,5.6,48.5553,13.0228,Solar,-1,NaN,Industry About,...,False,True,False,False,False,False,False,False,False,False
4,AUT,Austria,Jochenstein,132.0,48.5184,13.7076,Hydro,1956,Verbund,GEODB,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,DEU,Germany,W�lfersheim Solar Power Plant,5.3,50.3953,8.8275,Solar,-1,NaN,Industry About,...,False,True,False,False,False,False,False,False,False,False
1289,DEU,Germany,Heizkraftwerk Evonik Rheinfelden,16.0,47.5708,7.8072,Gas,1980,Evonik Degussa GmbH,Open Power System Data,...,False,False,True,False,False,False,False,False,False,False
1290,DEU,Germany,KW Rheinfelden,46.7,47.5699,7.8101,Hydro,2010,Energiedienst AG,Open Power System Data,...,True,False,False,False,False,False,False,False,False,False
1291,DEU,Germany,KW Wyhlen,37.9,47.5524,7.6581,Hydro,1912,Energiedienst AG,Open Power System Data,...,True,False,False,False,False,False,False,False,False,False
